In [1]:
import sys
sys.path.append('..')

from langchain.vectorstores import FAISS
from langchain.embeddings.azure_openai import AzureOpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os
from langchain.chat_models import AzureChatOpenAI

def load_env_variables(file_path):
    load_dotenv(file_path)
    print("Environment variables loaded successfully!")

env_file_path = "../.env"
load_env_variables = load_env_variables(env_file_path)


# embeddings = AzureOpenAIEmbeddings(azure_deployment=azure_deployment, openai_api_version=openai_api_version)
embeddings =  AzureOpenAIEmbeddings(
        deployment=os.getenv("EMB_DEPLOYMENT"),
        openai_api_version=os.getenv("EMB_OPENAI_API_VERSION"),
        model=os.getenv("EMB_MODEL"),
        openai_api_key=os.getenv("EMB_OPENAI_API_KEY"),
        openai_api_base=os.getenv("EMB_OPENAI_ENDPOINT"),
        openai_api_type=os.getenv("EMB_API_TYPE"),
    )

llm_gpt = AzureChatOpenAI(deployment_name=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME'), openai_api_version=os.getenv("OPENAI_API_VERSION"),
                        openai_api_base=os.getenv("OPENAI_API_BASE"), 
                        openai_api_type= os.getenv("OPENAI_API_TYPE"),
                        openai_api_key=os.getenv("OPENAI_API_KEY"),
                        max_tokens=3500,
                        temperature=0.0)
# for key, value in os.environ.items():
#   print(f"{key}: {value}")

Environment variables loaded successfully!


/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:113: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://polarisopenai.openai.azure.com/ to https://polarisopenai.openai.azure.com//openai.
  warnings.warn(
/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_community/embeddings/azure_openai.py:1

In [2]:
from src.utils.cube_semantic_custom import CubeSemanticLoader
cube_api_url = os.getenv("CUBE_API_URL")
cube_token = os.getenv("CUBE_TOKEN")
loader = CubeSemanticLoader(cube_api_url, cube_token, False)

def fetch_cube_metadata(local_vector_store_path, embeddings, *args, **kwargs):
    try:
        print("In upload_metadata to vector db")
        # # Load document from Cube meta api
        print(cube_api_url)
        loader = CubeSemanticLoader(cube_api_url, cube_token, False)
        documents = loader.load()
        print("Loaded documents: " + str(documents))
        vectorstore = FAISS.from_documents(documents, embeddings)
        vectorstore.save_local(local_vector_store_path)
        print("Vector store created and saved successfully!")
        return 1
    except Exception as e:
        # Handle exceptions gracefully and return an error response
        print("Error in upload_metadata: " + str(e))
        return 0


def load_vector_store(vector_store_path, embeddings, *args, **kwargs):
    # Load the vector store from the local file system
    vectorstore = FAISS.load_local(vector_store_path, embeddings, allow_dangerous_deserialization=True)
    print("Vector store loaded successfully!")
    
    return vectorstore


In [9]:
vector_store_path = "/Users/k.abhishek/Documents/experiments/metric_store/metric_store_gen_ai/data/vector_store/cube_meta_faiss_index"
fetch_cube_metadata(vector_store_path, embeddings)


In upload_metadata to vector db
http://localhost:4000/cubejs-api/v1
Loading metadata from http://localhost:4000/cubejs-api/v1...
Found 16 cube data objects in metadata.
Loading only companies, because it is a cube.
Processing companies...
SD: Processing companies.count : number...
SD: Dimension values: []
SD: Processing companies.id : number...
SD: Dimension values: []
SD: Processing companies.address : string...
SD: Dimension values: []
SD: Processing companies.email : string...
SD: Dimension values: []
SD: Processing companies.name : string...
SD: Dimension values: []
SD: Processing companies.phone : string...
SD: Dimension values: []
Loading only line_items, because it is a cube.
Processing line_items...
SD: Processing line_items.count : number...
SD: Dimension values: []
SD: Processing line_items.price : number...
SD: Dimension values: []
SD: Processing line_items.quantity : number...
SD: Dimension values: []
SD: Processing line_items.id : number...
SD: Dimension values: []
SD: Pro

1

In [11]:
vectorstore = load_vector_store(vector_store_path, embeddings)

In [ ]:
#Controller
from langchain import PromptTemplate
start_goal_prompt = PromptTemplate(
    template="""
    You are an very experienced and expert manager. You are given following query "{goal}".
    The query may contain different objectives which needs an expertise to solve. Your job is to provide a list of objectives that is easier for the experts to serve the query.
    Ensure objectives are streamlined to minimize the overall number of tasks while addressing the objective effectively. 
    Some part of the query may not be relevant to the objectives. You can ignore those parts.
    The number of tasks should be strictly less than 10.
    """,
    input_variables=["goal", "language", "tools"],
)

Tools

Metric Discovery: Assuming we are trying to 3 metrics 
Agent: Get relevant 
Get 3 top documents 

Tasks: Metric Discovery 
1. Rewrite the and Extract specifics of metric information for better retrieval
2. Extract top 3 documents related to metric information.
3. get information about metrics and compile the results and return 



In [13]:
import json
from crewai import Agent, Task
from langchain.tools import tool
#Will create below functions once the input is setup properly
# def create_vector_store

# def ingest_cube_meta():
  

class ExtractMetricInfo():
  @tool("Extract metric queried.")
  def extract_metric_info(query):
    """Useful to isolate metric information to be further used in the its discovery"""
    agent = Agent(
          role='Metric Isolator',
          goal=
          'Accurate Extraction of Metrics name',
          backstory=
          "You're a Metric isolator which is responsible for extracting the metrics name from the given content.",
          llm = llm_gpt,
          allow_delegation=False)
    task = Task(
          agent=agent,
          description=
          f' The primary objective is to ensure that all the metrics are correctly isolated and extracted from the query {query}. This extracted information should be organized in a form of list.',
          expected_output="some string",
    
      )
    extracted_metrics = task.execute()

    return extracted_metrics
  
  def get_similar_documents(query, vector_store):
    docs = vector_store.similarity_search(query)
    # take the first document as the best guess
    table_name = docs[0].metadata["table_name"]
    # Columns
    columns_question = "All available columns"
    column_docs = vector_store.similarity_search(
        columns_question,
        filter=dict(table_name=table_name),
        k=20
    )
    lines = []
    for column_doc in column_docs:
        column_title = column_doc.metadata["column_title"]
        column_name = column_doc.metadata["column_name"]
        column_data_type = column_doc.metadata["column_data_type"]
        lines.append(
            f"title: {column_title}, column name: {column_name}, datatype: {column_data_type}, member type: {column_doc.metadata['column_member_type']}"
        )
    columns = "\n".join(lines)
    print("columns :", columns)

In [14]:
query = "What is the most popular feature used by our paid subscribers"
extracted_metrics = ExtractMetricInfo.extract_metric_info(query)

/Users/k.abhishek/anaconda3/envs/agentgpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [16]:
type(extracted_metrics)

str